# CAO Points

In [1]:
# For dataframes
import pandas as pd

# Numerical arrays
import numpy as np
# For HTTP requests
import requests as rq

# For regular expressions
import re

# For accessing dates and times
import datetime as dt

# Accessing and downloading using urls
import urllib.request as urlrq

# For reading pdfs 
import tabula

### Datetime

In [2]:
"""Using datetime to create a variable that will refer to a string stating the current time.
This will be used throughout this notebook to save files with the current time in the filename."""

# Access the current date and time.
now = dt.datetime.now()

# Formatting the current date and time as a string and saving to a variable.
current_time = now.strftime('%Y%m%d_%H%M%S')

In [3]:
# Make dataframe scrollable to eyeball any obvious errors in dataframes
# Commenting out for now as it's slowing down the browser too much
# pd.set_option("display.max_rows", None)

### Functions

In [4]:
# A function that adds courses with specific symbols in the points columns to lists
# Then returns the points without the symbols
# * - Not all on this points score were offered places
# # - Test / Interview / Portfolio / Audition
# AQA - All qualified applicants
# Commented out for now until it's tested fully
"""
def points_to_array(s):
    portfolio = ''
    if s[0] == '#':
        portfolio = '#'
    random = ''
    if s[-1] == '*':
        random = '*'
    aqa = ''
    if s == 'AQA':
        aqa = 'AQA'
    points = ''
    for i in s:
        if i.isdigit():
            points = points + i
    return[points, portfolio, random, aqa]
"""

"\ndef points_to_array(s):\n    portfolio = ''\n    if s[0] == '#':\n        portfolio = '#'\n    random = ''\n    if s[-1] == '*':\n        random = '*'\n    aqa = ''\n    if s == 'AQA':\n        aqa = 'AQA'\n    points = ''\n    for i in s:\n        if i.isdigit():\n            points = points + i\n    return[points, portfolio, random, aqa]\n"

In [5]:
# A function that finds courses that required Test / Interview / Portfolio / Audition
# As indicated by the '#' character
def portfolio(df):
    # Find '#' in pointsR1 column
    df_portfolio1 = df[df['pointsR1'].str.match("#", na=False)]
    # find '#' in pointsR2 column
    df_portfolio2 = df[df['pointsR2'].str.match("#", na=False)]
    # Concat the dataframes together
    df_portfolio = pd.concat([df_portfolio1, df_portfolio2], ignore_index=True)
    # Remove duplicates
    df_portfolio = df_portfolio.drop_duplicates()
    return df_portfolio

In [6]:
# A function that finds courses where all on this points score were offered places
# As indicated by the '*' character
def random(df):
    # Find '*' in pointsR1 column
    df_random1 = df[df['pointsR1'].str[-1] == '*']
    # find '#' in pointsR2 column
    df_random2 = df[df['pointsR2'].str[-1] == '*']
    # Concat the dataframes together
    df_random = pd.concat([df_random1, df_random2], ignore_index=True)
    # Remove duplicates
    df_random = df_random.drop_duplicates()
    return df_random

In [7]:
# A function that finds courses where all qualified applicants earned places
# As indicated by AQA
def aqa(df):
    # Find 'AQA' in pointsR1 column
    df_aqa1 = df[df['pointsR1'].str.match("AQA", na=False)]
    # find 'AQA' in pointsR2 column
    df_aqa2 = df[df['pointsR2'].str.match("AQA", na=False)]
    # Concat the dataframes together
    df_aqa = pd.concat([df_aqa1, df_aqa2], ignore_index=True)
    # Remove duplicates
    df_aqa = df_aqa.drop_duplicates()
    return df_aqa

In [8]:
# A function that finds courses where there's new competition for available places
# As indicated by the 'v' character
def comp(df):
    # Find 'v' in pointsR1 column
    df_comp1 = df[df['pointsR1'].str[-1] == 'v']
    # find '#' in pointsR2 column
    df_comp2 = df[df['pointsR2'].str[-1] == 'v']
    # Concat the dataframes together
    df_comp = pd.concat([df_comp1, df_comp2], ignore_index=True)
    # Remove duplicates
    df_comp = df_comp.drop_duplicates()
    return df_comp

In [9]:
# Add new columns with point totals sans additional characters
# https://stackoverflow.com/questions/13682044/remove-unwanted-parts-from-strings-in-a-column/22238380
def points(df):
    df2 = df.assign(Round_1_Points=df['pointsR1'].str.replace(r'\D', '', regex=True))
    df3 = df2.assign(Round_2_Points=df['pointsR2'].str.replace(r'\D', '', regex=True))
    return df3

## 2021 Level 8 Points

In [10]:
resp2021_l8 = rq.get('http://www2.cao.ie/points/l8.php')

In [11]:
resp2021_l8

<Response [200]>

In [12]:
"""Have to change the encoding as the following error is returned:
'charmap' codec can't encode character '\x96' in position 25767: character maps to <undefined>"""

# The server uses the wrong encoding.
original_encoding = resp2021_l8.encoding

# Change to cp1252, which recognises the '\x96' character.
resp2021_l8.encoding = 'cp1252'

In [13]:
# Create a file path for the original data.
path2021_l8_html = 'cao-data/2021-points/2021-level-8-points/cao2021_level8_' + current_time + '.html'

In [14]:
# Save the original html file.
with open(path2021_l8_html, 'w') as f:
    f.write(resp2021_l8.text)

In [15]:
# re adapted from:https://github.com/ianmcloughlin/cao-points/blob/main/cao-points-analysis.ipynb

# Using regular expression to extract the lines of data we want.
re_course = re.compile(r'([A-Z]{2}[0-9]{3})(.*)')

In [16]:
# Code adapted from: https://github.com/ianmcloughlin/cao-points/blob/main/cao-points-analysis.ipynb


# The file path for the csv file.
path2021_l8 = 'cao-data/2021-points/2021-level-8-points/cao2021_level8_csv_' + current_time + '.csv'

# Keep track of how many courses we process.
no_lines = 0

# Open the csv file for writing.
with open(path2021_l8, 'w') as f:
    # Write a header row.
    f.write(','.join(['code', 'title', 'pointsR1', 'pointsR2']) + '\n')
    # Loop through lines of the response.
    for line in resp2021_l8.iter_lines():
        # Decode the line, using the wrong encoding!
        dline = line.decode('cp1252')
        # Match only the lines representing courses.
        if re_course.fullmatch(dline):
            # Add one to the lines counter.
            no_lines = no_lines + 1
            # The course code.
            course_code = dline[:5]
            # The course title.
            course_title = dline[7:57].strip()
            # Round one points.
            course_points = re.split(' +', dline[60:])
            if len(course_points) != 2:
                course_points = course_points[:2]
            # Join the fields using a comma.
            linesplit = [course_code, course_title, course_points[0], course_points[1]]
            # Rejoin the substrings with commas in between.
            f.write(','.join(linesplit) + '\n')

# Print the total number of processed lines.
print(f"Total number of lines is {no_lines}.")

Total number of lines is 949.


In [17]:
# Load the 2021 level 8 data to a pandas dataframe
df2021_l8 = pd.read_csv(path2021_l8, encoding='cp1252')

In [18]:
# Extracting all Portfolio/Test/Interview/Audition courses 
df2021_l8_portfolio = portfolio(df2021_l8)

In [19]:
df2021_l8_portfolio

,code,title,pointsR1,pointsR2
0,AL861,Animation and Illustration (portfolio),#575,NaN
1,AL863,Graphic and Digital Design (portfolio),#747,NaN
2,CW038,Art (portfolio),#700,NaN
3,CW858,Sports Management and Coaching (options portf...,#700,NaN
4,CR121,Music at CIT Cork School of Music,#904,#904
...,...,...,...,...
78,SG244,Fine Art,#501,NaN
79,TL801,Animation Visual Effects and Motion Design (L...,#718,NaN
80,WD027,Music,#321,NaN
96,MH802,Community and Youth Work (part-time in service),NaN,#


In [20]:
# Extracting courses where not everyone with those points were offered places
df2021_l8_random = random(df2021_l8)

In [21]:
df2021_l8_random

,code,title,pointsR1,pointsR2
0,AL830,General Nursing,451*,444
1,AL832,Mental Health Nursing,440*,431
2,AL870,Applied Psychology,484*,467*
3,CR320,Biomedical Science - Offered jointly by CIT an...,590*,NaN
4,CR930,Home Economics and Business,518*,NaN
...,...,...,...,...
118,DN600,Law (options),576,567*
119,DN660,Commerce International (options),555,554*
120,DN710,Economics,534,532*
123,LM090,Physical Education with concurrent Teacher Edu...,552,544*


In [22]:
# Extracting courses where all qualified applicants earned places
df2021_l8_aqa = aqa(df2021_l8)

In [23]:
df2021_l8_aqa

,code,title,pointsR1,pointsR2
0,MH801,Early Childhood - Teaching and Learning (part-...,AQA,AQA


In [24]:
# Add extra columns for points only using previously defined function
df2021_l8 = points(df2021_l8)

In [25]:
df2021_l8

,code,title,pointsR1,pointsR2,Round_1_Points,Round_2_Points
0,AL801,Software Design for Virtual Reality and Gaming,300,NaN,300,NaN
1,AL802,Software Design in Artificial Intelligence for...,313,NaN,313,NaN
2,AL803,Software Design for Mobile Apps and Connected ...,350,NaN,350,NaN
3,AL805,Computer Engineering for Network Infrastructure,321,NaN,321,NaN
4,AL810,Quantity Surveying,328,NaN,328,NaN
...,...,...,...,...,...,...
944,WD211,Creative Computing,270,NaN,270,NaN
945,WD212,Recreation and Sport Management,262,NaN,262,NaN
946,WD230,Mechanical and Manufacturing Engineering,230,230,230,230
947,WD231,Early Childhood Care and Education,266,NaN,266,NaN


In [26]:
# Rename columns to match later dataframes
df2021_l8.rename(columns={'code': 'Course Code', 'title': 'INSTITUTION and COURSE'}, inplace=True)

In [27]:
df2021_l8

,Course Code,INSTITUTION and COURSE,pointsR1,pointsR2,Round_1_Points,Round_2_Points
0,AL801,Software Design for Virtual Reality and Gaming,300,NaN,300,NaN
1,AL802,Software Design in Artificial Intelligence for...,313,NaN,313,NaN
2,AL803,Software Design for Mobile Apps and Connected ...,350,NaN,350,NaN
3,AL805,Computer Engineering for Network Infrastructure,321,NaN,321,NaN
4,AL810,Quantity Surveying,328,NaN,328,NaN
...,...,...,...,...,...,...
944,WD211,Creative Computing,270,NaN,270,NaN
945,WD212,Recreation and Sport Management,262,NaN,262,NaN
946,WD230,Mechanical and Manufacturing Engineering,230,230,230,230
947,WD231,Early Childhood Care and Education,266,NaN,266,NaN


In [28]:
# Save pandas dataframe to disk.
df2021_l8.to_csv(path2021_l8)

## 2021 Level 7/6 Points

In [29]:
resp2021_l76 = rq.get('http://www2.cao.ie/points/l76.php')

In [30]:
resp2021_l76

<Response [200]>

In [31]:
# Create a file path for the original data.
path2021_l76_html = 'cao-data/2021-points/2021-level-7-and-6-points/cao2021_level7_6_' + current_time + '.html'

In [32]:
# Save the original html file.
with open(path2021_l76_html, 'w') as f:
    f.write(resp2021_l76.text)

In [33]:
# The file path for the csv file.
path2021_l76 = 'cao-data/2021-points/2021-level-7-and-6-points/cao2021_level7_6_csv_' + current_time + '.csv'

# Keep track of how many courses we process.
no_lines = 0

# Open the csv file for writing.
with open(path2021_l76, 'w') as f:
    # Write a header row.
    f.write(','.join(['code', 'title', 'pointsR1', 'pointsR2']) + '\n')
    # Loop through lines of the response.
    for line in resp2021_l76.iter_lines():
        # Decode the line, using the wrong encoding!
        dline = line.decode('cp1252')
        # Match only the lines representing courses.
        if re_course.fullmatch(dline):
            # Add one to the lines counter.
            no_lines = no_lines + 1
            # The course code.
            course_code = dline[:5]
            # The course title.
            course_title = dline[7:57].strip()
            # Round one points.
            course_points = re.split(' +', dline[60:])
            if len(course_points) != 2:
                course_points = course_points[:2]
            # Join the fields using a comma.
            linesplit = [course_code, course_title, course_points[0], course_points[1]]
            # Rejoin the substrings with commas in between.
            f.write(','.join(linesplit) + '\n')

# Print the total number of processed lines.
print(f"Total number of lines is {no_lines}.")

Total number of lines is 416.


In [34]:
# Load the 2021 level 7 and level 6 data to a pandas dataframe
df2021_l76 = pd.read_csv(path2021_l76, encoding='cp1252')

In [35]:
df2021_l76

,code,title,pointsR1,pointsR2
0,AL605,Music and Instrument Technology,211,NaN
1,AL630,Pharmacy Technician,308,NaN
2,AL631,Dental Nursing,311,NaN
3,AL632,Applied Science,297,NaN
4,AL650,Business,AQA,AQA
...,...,...,...,...
411,WD188,Applied Health Care,220,NaN
412,WD205,Molecular Biology with Biopharmaceutical Science,AQA,262v
413,WD206,Electronic Engineering,180,NaN
414,WD207,Mechanical Engineering,172,NaN


In [36]:
# Extracting all Portfolio/Test/Interview/Audition courses 
df2021_l76_portfolio = portfolio(df2021_l76)

In [37]:
df2021_l76_portfolio

,code,title,pointsR1,pointsR2
0,AL763,Graphic Design (portfolio),#642,NaN
1,CW057,Art (portfolio),#700,NaN
2,CW807,Sport Coaching and Business Management - GAA (...,#700,NaN
3,CW817,Sport Coaching and Business Management - Rugby...,#700,NaN
4,CW827,Sport Coaching and Business Management - Socce...,#700,NaN
5,TU655,Computing (Networking Technologies) Learn and ...,#502,NaN
6,TU722,Process Instrumentation and Automation Learn a...,#346,#346
7,TU795,Visual Merchandising and Display,#618,NaN
8,TR802,Dental Hygiene,#577,#565
9,TR803,Dental Technology,#543,#498*


In [38]:
# Extracting courses where not everyone with those points were offered places
df2021_l76_random = random(df2021_l76)

In [39]:
df2021_l76_random

,code,title,pointsR1,pointsR2
0,LY847,Veterinary Nursing,389*,NaN
1,TR803,Dental Technology,#543,#498*


In [40]:
# Extracting courses where all qualified applicants earned places
df2021_l76_aqa = aqa(df2021_l76)

In [41]:
df2021_l76_aqa

,code,title,pointsR1,pointsR2
0,AL650,Business,AQA,AQA
1,AL660,Culinary Arts,AQA,AQA
2,AL661,Bar Supervision,AQA,NaN
3,AL663,Business (Sport and Recreation),AQA,AQA
4,AL761,Hotel and Leisure Management,AQA,AQA
5,TU772,Business (options),AQA,250v
6,TU792,Community and Youth Development,AQA,242v
7,WD003,Business,AQA,AQA
8,WD013,Legal Studies,AQA,206v
9,WD019,Recreation and Sport Management,AQA,AQA


In [42]:
# Extracting courses where there's new competition for available places
df2021_l76_comp = comp(df2021_l76)

In [43]:
df2021_l76_comp

,code,title,pointsR1,pointsR2
0,TU708,Engineering (Common Entry with Award options),117,263v
1,TU772,Business (options),AQA,250v
2,TU792,Community and Youth Development,AQA,242v
3,WD013,Legal Studies,AQA,206v
4,WD177,Science (Mol. Biology with Biopharm. Food Science,205,455v
5,WD205,Molecular Biology with Biopharmaceutical Science,AQA,262v


In [44]:
df2021_l76 = points(df2021_l76)

In [45]:
df2021_l76

,code,title,pointsR1,pointsR2,Round_1_Points,Round_2_Points
0,AL605,Music and Instrument Technology,211,NaN,211,NaN
1,AL630,Pharmacy Technician,308,NaN,308,NaN
2,AL631,Dental Nursing,311,NaN,311,NaN
3,AL632,Applied Science,297,NaN,297,NaN
4,AL650,Business,AQA,AQA,,
...,...,...,...,...,...,...
411,WD188,Applied Health Care,220,NaN,220,NaN
412,WD205,Molecular Biology with Biopharmaceutical Science,AQA,262v,,262
413,WD206,Electronic Engineering,180,NaN,180,NaN
414,WD207,Mechanical Engineering,172,NaN,172,NaN


In [46]:
# Rename columns to match later dataframes
df2021_l76.rename(columns={'code': 'Course Code', 'title': 'INSTITUTION and COURSE'}, inplace=True)

In [47]:
df2021_l76

,Course Code,INSTITUTION and COURSE,pointsR1,pointsR2,Round_1_Points,Round_2_Points
0,AL605,Music and Instrument Technology,211,NaN,211,NaN
1,AL630,Pharmacy Technician,308,NaN,308,NaN
2,AL631,Dental Nursing,311,NaN,311,NaN
3,AL632,Applied Science,297,NaN,297,NaN
4,AL650,Business,AQA,AQA,,
...,...,...,...,...,...,...
411,WD188,Applied Health Care,220,NaN,220,NaN
412,WD205,Molecular Biology with Biopharmaceutical Science,AQA,262v,,262
413,WD206,Electronic Engineering,180,NaN,180,NaN
414,WD207,Mechanical Engineering,172,NaN,172,NaN


In [48]:
# Save pandas data frame to disk.
df2021_l76.to_csv(path2021_l76)

## 2021 Level 8/7/6 Points Addendum

After the start of this project, the CAO website have released an excel file with additional 2021 points information. As of 13th of December 2021 they no longer link to the webpage with the points data, instead linking to the new excel file. The website used previously still appears to be live, and the code above will remain in the hopes they do not remove this link. It would be prudent to also load the excel file, however, not least because it contains additional points information such as EOS and Mid points values, which is the points information given in years prior to 2020. Including these values in the analysis will allow for a more valid comparison between the points values from all years.

In [49]:
# The points for levels 8, 7, and 6 were included in one spreadsheet on the CAO website
url2021_xlsx = 'http://www2.cao.ie/points/CAOPointsCharts2021.xlsx'

In [50]:
# Create a file path for the original data.
path2021_xlsx = 'cao-data/2021-points/2021-all-levels-points/cao2021_' + current_time + '.xlsx'

In [51]:
# Retrieve the excel file and save it 
urlrq.urlretrieve(url2021_xlsx, path2021_xlsx)

('cao-data/2021-points/2021-all-levels-points/cao2021_20211218_235228.xlsx',
 <http.client.HTTPMessage at 0x1ebbe48b2e0>)

https://stackoverflow.com/questions/65250207/pandas-cannot-open-an-excel-xlsx-file to fix an error 'XLRDError: Excel xlsx file; not supported'. 

In [52]:
# Load the 2021 data to a pandas dataframe
df2021_xlsx = pd.read_excel(url2021_xlsx, skiprows=11,  engine='openpyxl')

In [53]:
df2021_xlsx

,CATEGORY (ISCED Description),Course Title,Course Code,R1 Points,R1 Random,R2 Points,R2 Random,EOS Points,EOS Random,EOS Midpoints,Course Level,HEI,Test/Interview,AVP,v
0,Engineering and engineering trades,Music and Instrument Technology,AL605,211,NaN,NaN,NaN,211,NaN,319,6,Athlone Institute of Technology,NaN,NaN,NaN
1,Health,Pharmacy Technician,AL630,308,NaN,NaN,NaN,308,NaN,409,6,Athlone Institute of Technology,NaN,NaN,NaN
2,Health,Dental Nursing,AL631,311,NaN,NaN,NaN,311,NaN,400,6,Athlone Institute of Technology,NaN,NaN,NaN
3,Biological and related sciences,Applied Science,AL632,297,NaN,NaN,NaN,297,NaN,454,6,Athlone Institute of Technology,NaN,NaN,NaN
4,Business and administration,Business,AL650,AQA,NaN,AQA,NaN,AQA,NaN,351,6,Athlone Institute of Technology,NaN,avp,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1446,Information and Communication Technologies (ICTs),Creative Computing,WD211,270,NaN,NaN,NaN,270,NaN,392,8,Waterford Institute of Technology,NaN,NaN,NaN
1447,Personal services,Recreation and Sport Management,WD212,262,NaN,NaN,NaN,262,NaN,304,8,Waterford Institute of Technology,NaN,NaN,NaN
1448,Engineering and engineering trades,Mechanical and Manufacturing Engineering,WD230,230,NaN,230,NaN,230,NaN,361,8,Waterford Institute of Technology,NaN,avp,NaN
1449,Welfare,Early Childhood Care and Education,WD231,266,NaN,NaN,NaN,266,NaN,366,8,Waterford Institute of Technology,NaN,NaN,NaN


In [54]:
# Extract courses that required portfolio/test/interview/audition
df2021_portfolio_xslx = df2021_xlsx[df2021_xlsx['Test/Interview'].str.match("#", na=False)]
df2021_portfolio_xslx

,CATEGORY (ISCED Description),Course Title,Course Code,R1 Points,R1 Random,R2 Points,R2 Random,EOS Points,EOS Random,EOS Midpoints,Course Level,HEI,Test/Interview,AVP,v
26,Arts,Graphic Design (portfolio),AL763,642,NaN,NaN,NaN,642,NaN,811,7,Athlone Institute of Technology,#,NaN,NaN
56,Arts,Animation and Illustration (portfolio),AL861,575,NaN,NaN,NaN,575,NaN,760,8,Athlone Institute of Technology,#,NaN,NaN
57,Arts,Graphic and Digital Design (portfolio),AL863,747,NaN,NaN,NaN,747,NaN,808,8,Athlone Institute of Technology,#,NaN,NaN
65,Arts,Art (portfolio),CW038,#700,NaN,NaN,NaN,700,NaN,700,8,"Institute of Technology, Carlow",#,NaN,NaN
68,Arts,Art (portfolio),CW057,#700,NaN,NaN,NaN,700,NaN,700,7,"Institute of Technology, Carlow",#,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1306,Engineering and engineering trades,"Engineering in Mechatronic Systems (Interview,...",SG335,#,NaN,#,NaN,#,NaN,0,7,"Institute of Technology, Sligo",#,avp,NaN
1342,Arts,"Animation, Visual Effects and Motion Design (L...",TL701,NaN,NaN,NaN,NaN,NaN,NaN,0,7,"Institute of Technology, Tralee",#,NaN,NaN
1362,Arts,"Animation, Visual Effects and Motion Design (L...",TL801,718,NaN,NaN,NaN,718,NaN,823,8,"Institute of Technology, Tralee",#,NaN,NaN
1383,Welfare,Counselling with Addiction - Mature applicants...,TL889,NaN,NaN,NaN,NaN,NaN,NaN,0,8,"Institute of Technology, Tralee",#,NaN,NaN


In [55]:
# Extract courses where not everyone with that point total was offered a place
# Due to the different formatting, the previous function for this task cannot be reused
# Filters out the rows where the Random columns contain NaN values, leaving only '*' rows
df2021_randomr1 = df2021_xlsx[~df2021_xlsx['R1 Random'].isnull()]
df2021_randomr2 = df2021_xlsx[~df2021_xlsx['R2 Random'].isnull()]
df2021_random = pd.concat([df2021_randomr1, df2021_randomr2], ignore_index=True)
df2021_random = df2021_random.drop_duplicates()
df2021_random

,CATEGORY (ISCED Description),Course Title,Course Code,R1 Points,R1 Random,R2 Points,R2 Random,EOS Points,EOS Random,EOS Midpoints,Course Level,HEI,Test/Interview,AVP,v
0,Health,General Nursing,AL830,451,*,444,NaN,440,NaN,455,8,Athlone Institute of Technology,NaN,NaN,NaN
1,Health,Mental Health Nursing,AL832,440,*,431,NaN,431,NaN,451,8,Athlone Institute of Technology,NaN,NaN,NaN
2,Social and behavioural sciences,Applied Psychology,AL870,484,*,467,*,452,NaN,480,8,Athlone Institute of Technology,NaN,NaN,NaN
3,Biological and related sciences,Biomedical Science - Offered jointly by CIT an...,CR320,590,*,NaN,NaN,590,*,601,8,Cork Institute of Technology,NaN,NaN,NaN
4,Education,Home Economics and Business,CR930,518,*,NaN,NaN,510,*,542,8,Cork Institute of Technology,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,Law,Law (options),DN600,576,NaN,567,*,566,NaN,588,8,University College Dublin (NUI),NaN,NaN,NaN
128,Business and administration,Commerce International (options),DN660,555,NaN,554,*,554,NaN,567,8,University College Dublin (NUI),NaN,NaN,NaN
129,Social and behavioural sciences,Economics,DN710,534,NaN,532,*,522,NaN,544,8,University College Dublin (NUI),NaN,NaN,NaN
132,Education,Physical Education with concurrent Teacher Edu...,LM090,552,NaN,544,*,544,*,577,8,University of Limerick,NaN,NaN,NaN


In [56]:
# Rename columns to reuse AQA function
df2021_xlsx.rename(columns={'R1 Points': 'pointsR1', 'R2 Points ': 'pointsR2'}, inplace=True)

In [57]:
# Reuse function to find courses where all qualified applicants received offers
df2021_xlsx_aqa = aqa(df2021_xlsx)
df2021_xlsx_aqa

,CATEGORY (ISCED Description),Course Title,Course Code,pointsR1,R1 Random,pointsR2,R2 Random,EOS Points,EOS Random,EOS Midpoints,Course Level,HEI,Test/Interview,AVP,v
0,Business and administration,Business,AL650,AQA,NaN,AQA,NaN,AQA,NaN,351,6,Athlone Institute of Technology,NaN,avp,NaN
1,Personal services,Culinary Arts,AL660,AQA,NaN,AQA,NaN,AQA,NaN,271,6,Athlone Institute of Technology,NaN,avp,NaN
2,Personal services,Bar Supervision,AL661,AQA,NaN,NaN,NaN,153,NaN,279,6,Athlone Institute of Technology,NaN,avp,NaN
3,Personal services,Business (Sport and Recreation),AL663,AQA,NaN,AQA,NaN,AQA,NaN,311,6,Athlone Institute of Technology,NaN,avp,NaN
4,Personal services,Hotel and Leisure Management,AL761,AQA,NaN,AQA,NaN,AQA,NaN,306,7,Athlone Institute of Technology,NaN,avp,NaN
5,Engineering and engineering trades,Engineering (Common Entry with Award options),TU708,AQA,NaN,263,NaN,263,NaN,384,7,Technological University Dublin,NaN,avp,v
6,Business and administration,Business (options),TU772,AQA,NaN,250,NaN,AQA,NaN,408,7,Technological University Dublin,NaN,avp,v
7,Social and behavioural sciences,Community and Youth Development,TU792,AQA,NaN,242,NaN,AQA,NaN,310,7,Technological University Dublin,NaN,avp,v
8,Education,Early Childhood - Teaching and Learning (part-...,MH801,AQA,NaN,AQA,NaN,AQA,NaN,319,8,Maynooth University,NaN,avp,NaN
9,Business and administration,Business,WD003,AQA,NaN,AQA,NaN,AQA,NaN,367,6,Waterford Institute of Technology,NaN,avp,NaN


In [58]:
# Create a file path for the pandas data.
path2021_all = 'cao-data/2021-points/2021-all-levels-points/cao2021_' + current_time + '.csv'

In [59]:
# Save pandas data frame to disk.
df2021_xlsx.to_csv(path2021_all)

In [60]:
# Split 2021 df into df with l8 courses
df2021_xlsx_l8 = df2021_xlsx[df2021_xlsx['Course Level'] == 8]
df2021_xlsx_l8

,CATEGORY (ISCED Description),Course Title,Course Code,pointsR1,R1 Random,pointsR2,R2 Random,EOS Points,EOS Random,EOS Midpoints,Course Level,HEI,Test/Interview,AVP,v
29,Information and Communication Technologies (ICTs),Software Design for Virtual Reality and Gaming,AL801,300,NaN,NaN,NaN,300,NaN,359,8,Athlone Institute of Technology,NaN,NaN,NaN
30,Information and Communication Technologies (ICTs),Software Design in Artificial Intelligence for...,AL802,313,NaN,NaN,NaN,313,NaN,381,8,Athlone Institute of Technology,NaN,NaN,NaN
31,Information and Communication Technologies (ICTs),Software Design for Mobile Apps and Connected ...,AL803,350,NaN,NaN,NaN,350,NaN,398,8,Athlone Institute of Technology,NaN,NaN,NaN
32,Information and Communication Technologies (ICTs),Computer Engineering for Network Infrastructure,AL805,321,NaN,NaN,NaN,321,NaN,381,8,Athlone Institute of Technology,NaN,NaN,NaN
33,Architecture and construction,Quantity Surveying,AL810,328,NaN,NaN,NaN,328,NaN,377,8,Athlone Institute of Technology,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1446,Information and Communication Technologies (ICTs),Creative Computing,WD211,270,NaN,NaN,NaN,270,NaN,392,8,Waterford Institute of Technology,NaN,NaN,NaN
1447,Personal services,Recreation and Sport Management,WD212,262,NaN,NaN,NaN,262,NaN,304,8,Waterford Institute of Technology,NaN,NaN,NaN
1448,Engineering and engineering trades,Mechanical and Manufacturing Engineering,WD230,230,NaN,230,NaN,230,NaN,361,8,Waterford Institute of Technology,NaN,avp,NaN
1449,Welfare,Early Childhood Care and Education,WD231,266,NaN,NaN,NaN,266,NaN,366,8,Waterford Institute of Technology,NaN,NaN,NaN


In [61]:
# Create a file path for the pandas data.
path2021_xlsx_l8 = 'cao-data/2021-points/2021-level-8-points/cao2021_xlsx_l8_' + current_time + '.csv'

In [62]:
# Save pandas data frame to disk.
df2021_xlsx_l8.to_csv(path2021_xlsx_l8)

In [63]:
# Split 2021 df into df with l7/6 courses
df2021_xlsx_l76 = df2021_xlsx[df2021_xlsx['Course Level'] <= 7]
df2021_xlsx_l76

,CATEGORY (ISCED Description),Course Title,Course Code,pointsR1,R1 Random,pointsR2,R2 Random,EOS Points,EOS Random,EOS Midpoints,Course Level,HEI,Test/Interview,AVP,v
0,Engineering and engineering trades,Music and Instrument Technology,AL605,211,NaN,NaN,NaN,211,NaN,319,6,Athlone Institute of Technology,NaN,NaN,NaN
1,Health,Pharmacy Technician,AL630,308,NaN,NaN,NaN,308,NaN,409,6,Athlone Institute of Technology,NaN,NaN,NaN
2,Health,Dental Nursing,AL631,311,NaN,NaN,NaN,311,NaN,400,6,Athlone Institute of Technology,NaN,NaN,NaN
3,Biological and related sciences,Applied Science,AL632,297,NaN,NaN,NaN,297,NaN,454,6,Athlone Institute of Technology,NaN,NaN,NaN
4,Business and administration,Business,AL650,AQA,NaN,AQA,NaN,AQA,NaN,351,6,Athlone Institute of Technology,NaN,avp,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1434,Health,Applied Health Care,WD188,220,NaN,NaN,NaN,220,NaN,402,7,Waterford Institute of Technology,NaN,NaN,NaN
1441,Biological and related sciences,Molecular Biology with Biopharmaceutical Science,WD205,AQA,NaN,262,NaN,AQA,NaN,434,7,Waterford Institute of Technology,NaN,avp,NaN
1442,Engineering and engineering trades,Electronic Engineering,WD206,180,NaN,NaN,NaN,180,NaN,367,7,Waterford Institute of Technology,NaN,NaN,NaN
1443,Engineering and engineering trades,Mechanical Engineering,WD207,172,NaN,NaN,NaN,172,NaN,356,7,Waterford Institute of Technology,NaN,NaN,NaN


In [64]:
# Create a file path for the pandas data.
path2021_xlsx_l76 = 'cao-data/2021-points/2021-level-7-and-6-points/cao2021_xlsx_l76_' + current_time + '.csv'

In [65]:
# Save pandas data frame to disk.
df2021_xlsx_l76.to_csv(path2021_xlsx_l76)

It's worth noting that the excel file appears to have a greater number of courses listed than the original 2021 webpage.

## 2020 Level 8/7/6 Points

In [66]:
# The points for levels 8, 7, and 6 were included in one spreadsheet on the CAO website
url2020 = 'http://www2.cao.ie/points/CAOPointsCharts2020.xlsx'

In [67]:
# Create a file path for the original data.
path2020_xlsx = 'cao-data/2020-points/2020-all-levels-points/cao2020_' + current_time + '.xlsx'

In [68]:
# Retrieve the excel file and save it 
urlrq.urlretrieve(url2020, path2020_xlsx)

('cao-data/2020-points/2020-all-levels-points/cao2020_20211218_235228.xlsx',
 <http.client.HTTPMessage at 0x1ebbec1ed90>)

In [69]:
# Load the 2020 data to a pandas dataframe
df2020 = pd.read_excel(url2020, skiprows=10,  engine='openpyxl')

In [70]:
df2020

,CATEGORY (i.e.ISCED description),COURSE TITLE,COURSE CODE2,R1 POINTS,R1 Random *,R2 POINTS,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,avp,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,Business and administration,International Business,AC120,209,NaN,NaN,NaN,209,NaN,280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Humanities (except languages),Liberal Arts,AC137,252,NaN,NaN,NaN,252,NaN,270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arts,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arts,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arts,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,Manufacturing and processing,Manufacturing Engineering,WD208,188,NaN,NaN,NaN,188,NaN,339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1460,Information and Communication Technologies (ICTs),Software Systems Development,WD210,279,NaN,NaN,NaN,279,NaN,337,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1461,Information and Communication Technologies (ICTs),Creative Computing,WD211,271,NaN,NaN,NaN,271,NaN,318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1462,Personal services,Recreation and Sport Management,WD212,270,NaN,NaN,NaN,270,NaN,349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
# Rename columns to match later dataframes
df2020.rename(columns={'COURSE CODE2': 'Course Code', 'COURSE TITLE': 'Course Title',
                      'R1 POINTS': 'pointsR1', 'R2 POINTS': 'pointsR2'}, inplace=True)


In [72]:
df2020

,CATEGORY (i.e.ISCED description),Course Title,Course Code,pointsR1,R1 Random *,pointsR2,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,avp,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,Business and administration,International Business,AC120,209,NaN,NaN,NaN,209,NaN,280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Humanities (except languages),Liberal Arts,AC137,252,NaN,NaN,NaN,252,NaN,270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arts,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arts,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arts,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,Manufacturing and processing,Manufacturing Engineering,WD208,188,NaN,NaN,NaN,188,NaN,339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1460,Information and Communication Technologies (ICTs),Software Systems Development,WD210,279,NaN,NaN,NaN,279,NaN,337,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1461,Information and Communication Technologies (ICTs),Creative Computing,WD211,271,NaN,NaN,NaN,271,NaN,318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1462,Personal services,Recreation and Sport Management,WD212,270,NaN,NaN,NaN,270,NaN,349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
# Extract courses that required portfolio/test/interview/audition
df2020_portfolio = df2020[df2020['Test/Interview #'].str.match("#", na=False)]

In [74]:
df2020_portfolio

,CATEGORY (i.e.ISCED description),Course Title,Course Code,pointsR1,R1 Random *,pointsR2,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,avp,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
2,Arts,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arts,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arts,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Education,Education & Design or Fine Art (Second Level T...,AD202,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Arts,Fine Art (portfolio),AD204,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1382,Arts,Design - Visual Communication,TU973,717,NaN,NaN,NaN,717,NaN,865,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1383,Arts,Fine Art,TU974,709,NaN,563,NaN,563,NaN,826,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1384,Arts,Interior Design,TU975,632,NaN,NaN,NaN,632,NaN,879,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1385,Arts,Photography,TU976,695,NaN,NaN,NaN,695,NaN,926,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
# Extract courses where not everyone with that point total was offered a place
# Due to the different formatting, the previous function for this task cannot be reused
# Filters out the rows where the Random columns contain NaN values, leaving only '*' rows
df2020_randomr1 = df2020[~df2020['R1 Random *'].isnull()]
df2020_randomr2 = df2020[~df2020['R2 Random*'].isnull()]
df2020_random = pd.concat([df2020_randomr1, df2020_randomr2], ignore_index=True)
df2020_random = df2020_random.drop_duplicates()

In [76]:
df2020_random

,CATEGORY (i.e.ISCED description),Course Title,Course Code,pointsR1,R1 Random *,pointsR2,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,avp,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,Health,Psychiatric Nursing,AL832,387,*,384,NaN,384,NaN,399,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Education,"Education, Home Economics and Religious Ed - w...",AS002,441,*,434,*,434,*,466,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Education,"Education, Home Economics and Irish - with con...",AS003,440,*,440,NaN,440,NaN,484,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Health,General Nursing,AS110,419,*,NaN,NaN,419,*,431,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Health,Intellectual Disability Nursing,AS130,336,*,328,NaN,317,NaN,339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,Law,Law,TR004,566,NaN,565,*,565,*,578,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,Social and behavioural sciences,Psychology,TR006,567,NaN,555,*,555,*,578,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,Engineering and engineering trades,Engineering,TR032,510,NaN,509,*,508,NaN,533,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,Business and administration,Business Studies and German,TR086,509,NaN,499,*,499,NaN,518,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
# Reuse function to find courses where all qualified applicants received offers
df2020_aqa = aqa(df2020)

In [78]:
df2020_aqa

,CATEGORY (i.e.ISCED description),Course Title,Course Code,pointsR1,R1 Random *,pointsR2,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,avp,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,Information and Communication Technologies (ICTs),Software Design,AL600,AQA,NaN,NaN,NaN,104,NaN,279,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Engineering and engineering trades,Computer Engineering,AL601,AQA,NaN,219,NaN,112,NaN,192,...,avp,v,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Engineering and engineering trades,Mechanical Engineering,AL602,AQA,NaN,NaN,NaN,260,NaN,424,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Architecture and construction,Civil Engineering,AL604,AQA,NaN,AQA,NaN,AQA,NaN,244,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Engineering and engineering trades,Music and Instrument Technology,AL605,AQA,NaN,AQA,NaN,154,NaN,306,...,avp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Personal services,Culinary Arts,AL660,AQA,NaN,AQA,NaN,112,NaN,270,...,avp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Personal services,Bar Supervision,AL661,AQA,NaN,AQA,NaN,101,NaN,241,...,avp,v,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Personal services,Business (Sport and Recreation),AL663,AQA,NaN,AQA,NaN,120,NaN,262,...,avp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Business and administration,Business,CW006,AQA,NaN,AQA,NaN,AQA,NaN,254,...,avp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Social and behavioural sciences,Applied Social Care,DB528,AQA,NaN,AQA,NaN,AQA,NaN,337,...,avp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
# Create a file path for the pandas data.
path2020 = 'cao-data/2020-points/2020-all-levels-points/cao2020_' + current_time + '.csv'

In [80]:
# Save pandas data frame to disk.
df2020.to_csv(path2020)

In [81]:
# Split 2020 df into df with l8 courses
df2020_l8 = df2020[df2020['LEVEL'] == 8]
df2020_l8

,CATEGORY (i.e.ISCED description),Course Title,Course Code,pointsR1,R1 Random *,pointsR2,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,avp,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,Business and administration,International Business,AC120,209,NaN,NaN,NaN,209,NaN,280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Humanities (except languages),Liberal Arts,AC137,252,NaN,NaN,NaN,252,NaN,270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arts,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arts,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arts,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,Arts,Arts (options),WD200,AQA,NaN,AQA,NaN,AQA,NaN,336,...,avp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1460,Information and Communication Technologies (ICTs),Software Systems Development,WD210,279,NaN,NaN,NaN,279,NaN,337,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1461,Information and Communication Technologies (ICTs),Creative Computing,WD211,271,NaN,NaN,NaN,271,NaN,318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1462,Personal services,Recreation and Sport Management,WD212,270,NaN,NaN,NaN,270,NaN,349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
# Create a file path for the pandas data.
path2020_l8 = 'cao-data/2020-points/2020-level-8-points/cao2020_l8_' + current_time + '.csv'

In [83]:
# Save pandas data frame to disk.
df2020_l8.to_csv(path2020_l8)

In [84]:
# Split 2020 df into df with l7/6 courses
df2020_l76 = df2020[df2020['LEVEL'] <= 7]
df2020_l76

,CATEGORY (i.e.ISCED description),Course Title,Course Code,pointsR1,R1 Random *,pointsR2,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,avp,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
12,Information and Communication Technologies (ICTs),Software Design,AL600,AQA,NaN,NaN,NaN,104,NaN,279,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,Engineering and engineering trades,Computer Engineering,AL601,AQA,NaN,219,NaN,112,NaN,192,...,avp,v,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,Engineering and engineering trades,Mechanical Engineering,AL602,AQA,NaN,NaN,NaN,260,NaN,424,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,Architecture and construction,Civil Engineering,AL604,AQA,NaN,AQA,NaN,AQA,NaN,244,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,Engineering and engineering trades,Music and Instrument Technology,AL605,AQA,NaN,AQA,NaN,154,NaN,306,...,avp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1449,Health,Applied Health Care,WD188,201,NaN,NaN,NaN,201,NaN,349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1456,Health,Molecular Biology with Biopharmaceutical Science,WD205,228,NaN,NaN,NaN,228,NaN,446,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1457,Engineering and engineering trades,Electronic Engineering,WD206,179,NaN,NaN,NaN,179,NaN,373,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1458,Engineering and engineering trades,Mechanical Engineering,WD207,198,NaN,NaN,NaN,198,NaN,365,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
# Create a file path for the pandas data.
path2020_l76 = 'cao-data/2020-points/2020-level-7-and-6-points/cao2020_l76_' + current_time + '.csv'

In [86]:
# Save pandas data frame to disk.
df2020_l76.to_csv(path2020_l76)

## 2019 Level 8 Points

In [87]:
# Request CAO website for 2019 level 8 url
url2019_l8 = 'http://www2.cao.ie/points/lvl8_19.pdf'

In [88]:
# Create a file path for the original data.
path2019_l8_pdf = 'cao-data/2019-points/2019-level-8-points/cao2019_l8_' + current_time + '.pdf'

In [89]:
# Save pdf to disk
urlrq.urlretrieve(url2019_l8, path2019_l8_pdf)

('cao-data/2019-points/2019-level-8-points/cao2019_l8_20211218_235228.pdf',
 <http.client.HTTPMessage at 0x1ebbeb88eb0>)

In [90]:
# Read pdf into panda using tabula
# Using concat as tabula creates a new dataframe for each page of the pdf
# Takes pandas options to remove header as tabula assigns first row on each new page as the header 
df2019_l8_load = pd.concat(tabula.read_pdf("http://www2.cao.ie/points/lvl8_19.pdf", 
                                      pages = 'all', pandas_options={'header': None}))

In [91]:
# Have a look at the imported data
df2019_l8_load

,0,1,2,3
0,Course Code,INSTITUTION and COURSE,EOS,Mid
1,NaN,Athlone Institute of Technology,NaN,NaN
2,AL801,Software Design with Virtual Reality and Gaming,304,328
3,AL802,Software Design with Cloud Computing,301,306
4,AL803,Software Design with Mobile Apps and Connected...,309,337
...,...,...,...,...
37,WD200,Arts (options),221,296
38,WD210,Software Systems Development,271,329
39,WD211,Creative Computing,275,322
40,WD212,Recreation and Sport Management,274,311


In [92]:
# Set the first row of new dataframe as header
# https://www.codegrepper.com/code-examples/python/frameworks/django/change+header+to+first+row+pandas
df2019_l8_load.columns = df2019_l8_load.iloc[0]
df2019_l8_load = df2019_l8_load[1:]

In [93]:
df2019_l8_load

,Course Code,INSTITUTION and COURSE,EOS,Mid
1,NaN,Athlone Institute of Technology,NaN,NaN
2,AL801,Software Design with Virtual Reality and Gaming,304,328
3,AL802,Software Design with Cloud Computing,301,306
4,AL803,Software Design with Mobile Apps and Connected...,309,337
5,AL805,Network Management and Cloud Infrastructure,329,442
...,...,...,...,...
37,WD200,Arts (options),221,296
38,WD210,Software Systems Development,271,329
39,WD211,Creative Computing,275,322
40,WD212,Recreation and Sport Management,274,311


In [94]:
# Reset index as tabula had a new index for each page of the pdf
df2019_l8 = df2019_l8_load.reset_index()

In [95]:
# Delete rows that contain NaN in the Course Code column
df2019_l8 = df2019_l8.dropna(subset = ['Course Code'])

In [96]:
# Rename column to match df from other years
df2019_l8.rename(columns={'INSTITUTION and COURSE': 'Course Title'}, inplace=True)

In [97]:
# Extract rows where points contain a symbol and include them in new df
# All courses with # required Test / Interview / Portfolio / Audition
df2019_l8_portfolio = df2019_l8[df2019_l8['EOS'].str.match("#", na=False)]

In [98]:
df2019_l8_portfolio

,index,Course Code,Course Title,EOS,Mid
24,25,AL861,Animation and Illustration (portfolio),#615,899
25,26,AL863,Graphic and Digital Design (portfolio),#703,898
31,32,CW038,"Art (portfolio, Wexford)",#700,700
58,15,CW858,"Sports Management and Coaching (options, portf...",#700,700
74,31,CR121,Music at CIT Cork School of Music,#633,1052
...,...,...,...,...,...
858,45,MH802,"Community and Youth Work (part-time, in service",#,NaN
859,46,MH803,"Local Studies or Community Studies (part-time,...",#,NaN
879,11,SG244,Fine Art,#355,496
924,1,TL889,Counselling with Addiction - Mature applicants...,#,NaN


In [99]:
# Extracting courses where not everyone with those points were offered places
df2019_l8_random = df2019_l8[df2019_l8['EOS'].str[-1] == '*']

In [100]:
df2019_l8_random

,index,Course Code,Course Title,EOS,Mid
129,31,CK201,Commerce,465*,489
163,10,CK704,Occupational Therapy,532*,554
166,13,CK707,Medical and Health Sciences,510*,543
172,19,CK791,Medicine - Graduate Entry (GAMSAT required),#58*,59
179,26,CM001,Education - Primary Teaching,452*,462
211,3,DC002,Education - Primary Teaching,462*,485
452,24,RC004,Physiotherapy,532*,542
460,32,TR006,Psychology,555*,577
463,35,TR012,History and Political Science,532*,557
467,39,TR018,Law and French,532*,554


In [101]:
# Extracting courses where all qualified applicants earned places
df2019_l8_aqa = df2019_l8[df2019_l8['EOS'].str.match("AQA", na=False)]

In [102]:
df2019_l8_aqa

,index,Course Code,Course Title,EOS,Mid


In [103]:
# Add new column with EOS point numbers removing non-digit characters
df2019_l8 = df2019_l8.assign(EOS_Points=df2019_l8['EOS'].str.replace(r'\D', '', regex=True))

In [104]:
df2019_l8

,index,Course Code,Course Title,EOS,Mid,EOS_Points
1,2,AL801,Software Design with Virtual Reality and Gaming,304,328,304
2,3,AL802,Software Design with Cloud Computing,301,306,301
3,4,AL803,Software Design with Mobile Apps and Connected...,309,337,309
4,5,AL805,Network Management and Cloud Infrastructure,329,442,329
5,6,AL810,Quantity Surveying,307,349,307
...,...,...,...,...,...,...
960,37,WD200,Arts (options),221,296,221
961,38,WD210,Software Systems Development,271,329,271
962,39,WD211,Creative Computing,275,322,275
963,40,WD212,Recreation and Sport Management,274,311,274


Have to strip some characters from Mid points total, where the letters 'ic' appear before the points total, which appears to be due to '#+matic' entries in the pdf. For some reason, using str.replace only works on the first few rows of the dataframe, and after that every entry is left as NaN, therefore another method is used.

In [106]:
# Add new column with EOS mid point numbers removing non-digit characters
def try_extract(pattern, string):
    try:
        m = pattern.search(string)
        return m.group(0)
    except (TypeError, ValueError, AttributeError):
        return np.nan

p = re.compile(r'\d+')
df2019_l8['Mid'] = [try_extract(p, x) for x in df2019_l8['Mid']]


In [107]:
df2019_l8

,index,Course Code,Course Title,EOS,Mid,EOS_Points
1,2,AL801,Software Design with Virtual Reality and Gaming,304,328,304
2,3,AL802,Software Design with Cloud Computing,301,306,301
3,4,AL803,Software Design with Mobile Apps and Connected...,309,337,309
4,5,AL805,Network Management and Cloud Infrastructure,329,442,329
5,6,AL810,Quantity Surveying,307,349,307
...,...,...,...,...,...,...
960,37,WD200,Arts (options),221,NaN,221
961,38,WD210,Software Systems Development,271,NaN,271
962,39,WD211,Creative Computing,275,NaN,275
963,40,WD212,Recreation and Sport Management,274,NaN,274


In [ ]:
# Create a file path for the pandas data.
path2019_l8 = 'cao-data/2019-points/2019-level-8-points/cao2019_l8_' + current_time + '.csv'

In [ ]:
# Save pandas data frame to disk.
df2019_l8.to_csv(path2019_l8)

## 2019 Level 7/6 Points

In [ ]:
# Request CAO website for 2019 level 7 and 6 url
url2019_l76 = 'http://www2.cao.ie/points/lvl76_19.pdf'

In [ ]:
# Create a file path for the original data.
path2019_l76_pdf = 'cao-data/2019-points/2019-level-7-and-6-points/cao2019_l76_' + current_time + '.pdf'

In [ ]:
# Save pdf to disk
urlrq.urlretrieve(url2019_l76, path2019_l76_pdf)

In [ ]:
# Read pdf into panda using tabula
# Using concat as tabula creates a new dataframe for each page of the pdf
# Takes pandas options to remove header as tabula assigns first row on each new page as the header 
df2019_l76_load = pd.concat(tabula.read_pdf("http://www2.cao.ie/points/lvl76_19.pdf", 
                                      pages = 'all', pandas_options={'header': None}))

In [ ]:
# Have a look at the imported data
df2019_l76_load

In [ ]:
# Rename the columns to match the 2019 level 8 data
df2019_l76_load.columns = ['Course Code', 'Course Title', 'EOS', 'Mid']

In [ ]:
# With the 2019 level 8 data, tabula ignored the introductory information 
# But it is included in the 2019 level 7 and 6 data so we have to remove it
# Reusing regular expressions to extract rows with the course codes we want
df2019_l76 = df2019_l76_load[df2019_l76_load['Course Code'].str.match(re_course, na=False)]

In [ ]:
# Reset index as tabula had a new index for each page of the pdf
df2019_l76 = df2019_l76.reset_index()

In [ ]:
df2019_l76

In [ ]:
# Extract rows where points contain a symbol and include them in new df
# All courses with # required Test / Interview / Portfolio / Audition
df2019_l76_portfolio = df2019_l76[df2019_l76['EOS'].str.match("#", na=False)]

In [ ]:
df2019_l76_portfolio

In [ ]:
# Extracting courses where not everyone with those points were offered places
df2019_l76_random = df2019_l76[df2019_l76['EOS'].str[-1] == '*']

In [ ]:
df2019_l76_random

In [ ]:
# Extracting courses where all qualified applicants earned places
df2019_l76_aqa = df2019_l76[df2019_l76['EOS'].str.match("AQA", na=False)]

In [ ]:
df2019_l76_aqa

In [ ]:
# Add new column with EOS point numbers removing non-digit characters
df2019_l76 = df2019_l76.assign(EOS_Points=df2019_l76['EOS'].str.replace(r'\D', '', regex=True))

While the regex seems to work without issue with the other dataframes, and works for most of this dataframe, in this instance the final 6 entries did not have their points added to the new Points column, and instead these values were left NaN

In [ ]:
# View the 2019 l7/6 dataframe
df2019_l76

In [ ]:
# Replace the NaN values with the values from the EOS column
df2019_l76.EOS_Points.fillna(df2019_l76.EOS, inplace=True)
df2019_l76

In [ ]:
# View the 2019 level 7/6 dataframe
df2019_l76

In [ ]:
# Create a file path for the pandas data.
path2019_l76 = 'cao-data/2019-points/2019-level-7-and-6-points/cao2019_l76_' + current_time + '.csv'

In [ ]:
# Save pandas data frame to disk.
df2019_l76.to_csv(path2019_l76)

## 2018 Level 8 Points

In [ ]:
# Request CAO website for 2018 level 8 url
url2018_l8 = 'http://www2.cao.ie/points/lvl8_18.pdf'

In [ ]:
# Create a file path for the original data.
path2018_l8_pdf = 'cao-data/2018-points/2018-level-8-points/cao2018_l8_' + current_time + '.pdf'

In [ ]:
# Save pdf to disk
urlrq.urlretrieve(url2018_l8, path2018_l8_pdf)

In [ ]:
# Read pdf into panda using tabula
# Using concat as tabula creates a new dataframe for each page of the pdf
# Takes pandas options to remove header as tabula assigns first row on each new page as the header 
df2018_l8_load = pd.concat(tabula.read_pdf("http://www2.cao.ie/points/lvl8_18.pdf", 
                                      pages = 'all', pandas_options={'header': None}))

In [ ]:
df2018_l8_load

In [ ]:
# Rename the columns to match the 2019 level 8 data
df2018_l8_load.columns = ['Course Code', 'Course Title', 'EOS', 'Mid']

In [ ]:
# Need to filter out the introductory blurb that isn't needed in the dataframe
# Reusing regular expressions to extract rows with the course codes we want
df2018_l8 = df2018_l8_load[df2018_l8_load['Course Code'].str.match(re_course, na=False)]

In [ ]:
# Reset index as tabula had a new index for each page of the pdf
df2018_l8 = df2018_l8.reset_index()

In [ ]:
df2018_l8

In [ ]:
# Extract rows where points contain a symbol and include them in new df
# All courses with # required Test / Interview / Portfolio / Audition
df2018_l8_portfolio = df2018_l8[df2018_l8['EOS'].str.match("#", na=False)]

In [ ]:
df2018_l8_portfolio

In [ ]:
# Extracting courses where not everyone with those points were offered places
df2018_l8_random = df2018_l8[df2018_l8['EOS'].str[-1] == '*']

In [ ]:
df2018_l8_random

In [ ]:
# Extracting courses where all qualified applicants earned places
df2018_l8_aqa = df2018_l8[df2018_l8['EOS'].str.match("AQA", na=False)]

In [ ]:
df2018_l8_aqa

In [ ]:
# Add new column with EOS point numbers removing non-digit characters
df2018_l8 = df2018_l8.assign(EOS_Points=df2018_l8['EOS'].str.replace(r'\D', '', regex=True))

In [ ]:
df2018_l8

In [ ]:
# Create a file path for the pandas data.
path2018_l8 = 'cao-data/2018-points/2018-level-8-points/cao2018_l8_' + current_time + '.csv'

In [ ]:
# Save pandas data frame to disk.
df2018_l8.to_csv(path2018_l8)

## 2018 Level 7/6 Points

In [ ]:
# Request CAO website for 2018 level 7 and 6 url
url2018_l76 = 'http://www2.cao.ie/points/lvl76_18.pdf'

In [ ]:
# Create a file path for the original data.
path2018_l76_pdf = 'cao-data/2018-points/2018-level-7-and-6-points/cao2018_l76_' + current_time + '.pdf'

In [ ]:
# Save pdf to disk
urlrq.urlretrieve(url2018_l76, path2018_l76_pdf)

In [ ]:
# Read pdf into panda using tabula
# Using concat as tabula creates a new dataframe for each page of the pdf
# Takes pandas options to remove header as tabula assigns first row on each new page as the header 
df2018_l76_load = pd.concat(tabula.read_pdf("http://www2.cao.ie/points/lvl76_18.pdf", 
                                      pages = 'all', pandas_options={'header': None}))

In [ ]:
# Have a look at the imported data
df2018_l76_load

In [ ]:
# Rename the columns to match the 2019 level 8 data
df2018_l76_load.columns = ['Course Code', 'Course Title', 'EOS', 'Mid']

In [ ]:
# Reusing regular expressions to extract rows with the course codes we want
df2018_l76 = df2018_l76_load[df2018_l76_load['Course Code'].str.match(re_course, na=False)]

In [ ]:
# Reset index as tabula had a new index for each page of the pdf
df2018_l76 = df2018_l76.reset_index()

In [ ]:
df2018_l76

In [ ]:
# Extract rows where points contain a symbol and include them in new df
# All courses with # required Test / Interview / Portfolio / Audition
df2018_l76_portfolio = df2018_l76[df2018_l76['EOS'].str.match("#", na=False)]

In [ ]:
df2018_l76_portfolio

In [ ]:
# Extracting courses where not everyone with those points were offered places
df2018_l76_random = df2018_l76[df2018_l76['EOS'].str[-1] == '*']

In [ ]:
df2018_l76_random

In [ ]:
# Extracting courses where all qualified applicants earned places
df2018_l76_aqa = df2018_l76[df2018_l76['EOS'].str.match("AQA", na=False)]

In [ ]:
df2018_l76_aqa

In [ ]:
# Add new column with EOS point numbers removing non-digit characters
df2018_l76 = df2018_l76.assign(EOS_Points=df2018_l76['EOS'].str.replace(r'\D', '', regex=True))

In [ ]:
df2018_l76

In [ ]:
# Create a file path for the pandas data.
path2018_l76 = 'cao-data/2018-points/2018-level-7-and-6-points/cao2018_l76_' + current_time + '.csv'

In [ ]:
# Save pandas data frame to disk.
df2018_l76.to_csv(path2018_l76)

## Concat and join

## Level 8

The code below originally used the data taken from the old webpage of 2021 points data. However, since the later excel file of 2021 points has additional information and lists more courses, the code below has been altered to get the 2021 data from the excel file. 

In [ ]:
# Create a df of just the 2021 level 8 Course codes and titles
courses2021_l8 = df2021_xlsx_l8[['Course Code', 'Course Title']]
courses2021_l8

In [ ]:
# Create a df of just the 2020 level 8 Course codes and titles
courses2020_l8 = df2020_l8[['Course Code', 'Course Title']]
courses2020_l8

In [ ]:
# Create a df of just the 2019 level 8 Course codes and titles
courses2019_l8 = df2019_l8[['Course Code', 'Course Title']]
courses2019_l8

In [ ]:
# Create a df of just the 2018 level 8 Course codes and titles
courses2018_l8 = df2018_l8[['Course Code', 'Course Title']]
courses2018_l8

In [ ]:
# Concat level 8 courses df from 2021-2018 together
allcourses_l8 = pd.concat([courses2021_l8, courses2020_l8, 
                           courses2019_l8, courses2018_l8], 
                          ignore_index=True)
allcourses_l8

In [ ]:
# Sort all level 8 courses by course code
allcourses_l8.sort_values('Course Code')

In [ ]:
# Finds all extra copies of duplicated rows.
allcourses_l8[allcourses_l8.duplicated()]

In [ ]:
# Returns a copy of the data frame with duplciates removed - based only on code.
allcourses_l8.drop_duplicates(subset=['Course Code'], 
                              inplace=True, ignore_index=True)
allcourses_l8

### Join Level 8 points 

In [ ]:
# Set the index to the Course Code column.
df2021_xlsx_l8.set_index('Course Code', inplace=True)
df2021_xlsx_l8

In [ ]:
# Set the index to the Course Code column.
allcourses_l8.set_index('Course Code', inplace=True)

In [ ]:
# Join 2021 Level 8 points to all courses df
allcourses_l8 = allcourses_l8.join(df2021_xlsx_l8[['pointsR1', 
                                              'pointsR2', 'EOS Points',
                                             'EOS Midpoints']])

In [ ]:
# Rename Columns
allcourses_l8.columns = ['Course Title', 'points_r1_2021', 
                         'points_r2_2021', 'EOS 2021', 'EOS Mid 2021']
allcourses_l8

In [ ]:
# Extract level 8 points from 2020 dataframe
df2020_l8_points = df2020_l8[['Course Code', 'pointsR1', 
                              'pointsR2', 'EOS', 'EOS Mid-point']]
df2020_l8_points.columns = ['Course Code', 'points_r1_2020', 
                            'points_r2_2020', 'EOS 2020', 'EOS Mid 2020']
df2020_l8_points

In [ ]:
# Set the index to the Course Code column.
df2020_l8_points.set_index('Course Code', inplace=True)
df2020_l8_points

In [ ]:
# Join 2020 level 8 points to allcourses.
allcourses_l8 = allcourses_l8.join(df2020_l8_points)
allcourses_l8

In [ ]:
# Extract level 8 points from 2019 dataframe
df2019_l8_points = df2019_l8[['Course Code', 'EOS_Points', 'Mid_Points']]
df2019_l8_points.columns = ['Course Code', 'EOS 2019', 'EOS Mid 2019']
df2019_l8_points

In [ ]:
# Set the index to the Course Code column.
df2019_l8_points.set_index('Course Code', inplace=True)
df2019_l8_points

In [ ]:
# Join 2019 level 8 points to allcourses.
allcourses_l8 = allcourses_l8.join(df2019_l8_points)
allcourses_l8

In [ ]:
# Extract level 8 points from 2018 dataframe
df2018_l8_points = df2018_l8[['Course Code', 'EOS_Points','Mid']]
df2018_l8_points.columns = ['Course Code', 'EOS 2018', 'EOS Mid 2018']
df2018_l8_points

In [ ]:
# Set the index to the Course Code column.
df2018_l8_points.set_index('Course Code', inplace=True)
df2018_l8_points

In [ ]:
# Join 2018 level 8 points to allcourses.
allcourses_l8 = allcourses_l8.join(df2018_l8_points)
allcourses_l8

In [ ]:
# Remove remaining '#+matric' values from points columns
allcourses_l8 = allcourses_l8.replace('\#\+matric',np.NaN, regex=True)

In [ ]:
# Remove remaining hashtags from points columns 
allcourses_l8 = allcourses_l8.replace('#','', regex=True)

In [ ]:
# Add NaN values to blank cells
allcourses_l8 = allcourses_l8.replace('',np.NaN, regex=True)

In [ ]:
# Replace remaining AQA cells with NaN
allcourses_l8 = allcourses_l8.replace('AQA',np.NaN, regex=True)

In [ ]:
allcourses_l8

In [ ]:
# convert points columns to floats
colsl8 = allcourses_l8.columns[1:]
allcourses_l8[colsl8] = allcourses_l8[colsl8].apply(pd.to_numeric)

In [ ]:
# Check column types
allcourses_l8.dtypes

In [ ]:
# View all Level 8 dataframe
allcourses_l8

In [ ]:
# Create a file path for the all courses pandas data.
pathallcourses_l8 = 'cao-data/all-courses-points/all-courses-level-8-points/allcourses_l8_' + current_time + '.csv'

In [ ]:
# Save pandas data frame to disk.
allcourses_l8.to_csv(pathallcourses_l8)

## Concat Level 7/6

In [ ]:
# Create a df of just the 2021 level 7/6 Course codes and titles
courses2021_l76 = df2021_xlsx_l76[['Course Code', 'Course Title']]
courses2021_l76

In [ ]:
# Create a df of just the 2020 level 7/6 Course codes and titles
courses2020_l76 = df2020_l76[['Course Code', 'Course Title']]
courses2020_l76

In [ ]:
# Create a df of just the 2019 level 7/6 Course codes and titles
courses2019_l76 = df2019_l76[['Course Code', 'Course Title']]
courses2019_l76

In [ ]:
# Create a df of just the 2018 level 7/6 Course codes and titles
courses2018_l76 = df2018_l76[['Course Code', 'Course Title']]
courses2018_l76

In [ ]:
# Concat level 7/6 courses df from 2021-2018 together
allcourses_l76 = pd.concat([courses2021_l76, courses2020_l76, 
                            courses2019_l76, courses2018_l76], 
                           ignore_index=True)
allcourses_l76

In [ ]:
# Sort all level 7/6 courses by course code
allcourses_l76.sort_values('Course Code')

In [ ]:
# Finds all extra copies of duplicated rows.
allcourses_l76[allcourses_l76.duplicated()]

In [ ]:
# Returns a copy of the data frame with duplciates removed - based only on code.
allcourses_l76.drop_duplicates(subset=['Course Code'], 
                              inplace=True, ignore_index=True)
allcourses_l76

### Join Level 7/6 Points

In [ ]:
# Set the index to the Course Code column.
df2021_xlsx_l76.set_index('Course Code', inplace=True)
df2021_xlsx_l76

In [ ]:
# Set the index to the Course Code column.
allcourses_l76.set_index('Course Code', inplace=True)

In [ ]:
# Join 2021 Level 7/6 points to all courses df
allcourses_l76 = allcourses_l76.join(df2021_xlsx_l76[['pointsR1', 
                                              'pointsR2', 'EOS Points',
                                             'EOS Midpoints']])

In [ ]:
# Rename Columns
allcourses_l76.columns = ['Course Title', 'points_r1_2021', 
                         'points_r2_2021', 'EOS 2021', 'EOS Mid 2021']
allcourses_l76

In [ ]:
# Extract level 7/6 points from 2020 dataframe
df2020_l76_points = df2020_l76[['Course Code', 'pointsR1', 
                              'pointsR2', 'EOS', 'EOS Mid-point']]
df2020_l76_points.columns = ['Course Code', 'points_r1_2020', 
                            'points_r2_2020', 'EOS 2020', 'EOS Mid 2020']
df2020_l76_points

In [ ]:
# Set the index to the Course Code column.
df2020_l76_points.set_index('Course Code', inplace=True)
df2020_l76_points

In [ ]:
# Join 2020 level 7/6 points to allcourses.
allcourses_l76 = allcourses_l76.join(df2020_l76_points)
allcourses_l76

In [ ]:
# Extract level 7/6 points from 2019 dataframe
df2019_l76_points = df2019_l76[['Course Code', 'EOS_Points','Mid']]
df2019_l76_points.columns = ['Course Code', 'EOS 2019', 'EOS Mid 2019']
df2019_l76_points

In [ ]:
# Set the index to the Course Code column.
df2019_l76_points.set_index('Course Code', inplace=True)
df2019_l76_points

In [ ]:
# Join 2019 level 7/6 points to allcourses.
allcourses_l76 = allcourses_l76.join(df2019_l76_points)
allcourses_l76

In [ ]:
# Extract level 7/6 points from 2018 dataframe
df2018_l76_points = df2018_l76[['Course Code', 'EOS_Points','Mid']]
df2018_l76_points.columns = ['Course Code', 'EOS 2018', 'EOS Mid 2018']
df2018_l76_points

In [ ]:
# Set the index to the Course Code column.
df2018_l76_points.set_index('Course Code', inplace=True)
df2018_l76_points

In [ ]:
# Join 2018 level 7/6 points to allcourses.
allcourses_l76 = allcourses_l76.join(df2018_l76_points)
allcourses_l76

In [ ]:
# Remove remaining '#+matric' values from points columns
allcourses_l76 = allcourses_l76.replace('\#\+matric',np.NaN, regex=True)

In [ ]:
# Remove remaining hashtags from points columns 
allcourses_l76 = allcourses_l76.replace('#','', regex=True)

In [ ]:
# Add NaN values to blank cells
allcourses_l76 = allcourses_l76.replace('',np.NaN, regex=True)

In [ ]:
# Replace remaining AQA cells with NaN
allcourses_l76 = allcourses_l76.replace('AQA',np.NaN, regex=True)

In [ ]:
allcourses_l76

In [ ]:
# convert points columns to floats
colsl76 = allcourses_l76.columns[1:]
allcourses_l76[colsl76] = allcourses_l76[colsl76].apply(pd.to_numeric)

In [ ]:
# Check column types
allcourses_l76.dtypes

In [ ]:
# Create a file path for the all courses pandas data.
pathallcourses_l76 = 'cao-data/all-courses-points/all-courses-level-7-and-6-points/allcourses_l76_' + current_time + '.csv'

In [ ]:
# Save pandas data frame to disk.
allcourses_l76.to_csv(pathallcourses_l76)

## References